### 1. Install Packages

In [1]:
!pip install flask
!pip install flask-ngrok
print("Installed")

Installed


### 2. Install Ngrok

In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

--2022-05-09 18:00:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  37.5MB/s    in 0.3s    

2022-05-09 18:00:07 (37.5 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok


### 3. Ensure Ngrok is Installed

In [3]:
!./ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

### 4. Set Ngrok Token

In [4]:
!./ngrok authtoken 28vL9ko9ifzxsPY0BNrduIR4Wkw_3qAy4kCWkbzB9HEaz3VrK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### 5.Building Flask App

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Cap/Backend_ML') #/content/drive/MyDrive/'your path'

In [ ]:
from flask import Flask,request, jsonify, abort, make_response
from flask_ngrok import run_with_ngrok
from sentiment.sentiment_model import get_sentiment_model_class
from keyword_extractor.keyword_model import get_keyword_model_class
import yaml

app = Flask(__name__)
run_with_ngrok(app)

with open('./sentiment/configs/cfg.yaml') as f:
    sentiment_cfg = yaml.load(f, Loader=yaml.FullLoader)
with open('./keyword/configs/cfg.yaml') as f:
    keyword_cfg = yaml.load(f, Loader=yaml.FullLoader)
    
sentiment_model_class = get_sentiment_model_class(sentiment_cfg['model_name'])
sentiment_model = sentiment_model_class(sentiment_cfg)

keyword_model_class = get_keyword_model_class(keyword_cfg['model_name'])
keyword_model = keyword_model_class(keyword_cfg)

# 서버 작동 여부 확인 Route ('/')
@app.route('/')
def server_check():
    return jsonify(message="Server Linked!")

# Emotion Prediction Route ('/prediction')
@app.route('/prediction', methods = ['POST'])
def prediction():
    # request를 json으로 받아온다.
    req = request.get_json()
    content = req['content']
    
    
    emotion = sentiment_model.predict(content)
    hashtag = keyword_model.predict(content)

    return jsonify(emotion= emotion,hashtag_1 = hashtag[0],hashtag_2 = hashtag[1],hashtag_3 = hashtag[2])

@app.errorhandler(500)
def internal_server_error(e):
    abort(make_response(jsonify(message = "Internal Server Error"),500))

if __name__ == '__main__':
    app.run()
    app.register_error_handler(500,internal_server_error)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://87c9-34-68-90-254.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/May/2022 18:00:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:00:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:06:45] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:08:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:09:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:10:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:11:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:12:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:13:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:14:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:15:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:16:23] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:17:23] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 18:18:23] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 1